In [1]:
import pandas as pd
from splycer.blocker import BlockDB
from splycer.record_set import RecordDB
from splycer.pairs_set import PairsDB
from splycer.feature_engineer import FeatureEngineer
import recordlinkage as rl
import pyodbc
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import pickle as pkl
from tqdm import tqdm

import turbodbc
conn = turbodbc.connect('rec_db')

In [2]:
# Load the model
modelPath = r'R:\JoePriceResearch\record_linking\projects\deep_learning\paper_RR\CensusTree_2020\final\3-train_models\model_1900_1910.dat'
model = pkl.load(open(modelPath, "rb"))


In [3]:
extra_join = ''
sql1910 = RecordDB('compiled_1910','index','rec_db',extra_joins=extra_join)
sql1920 = RecordDB('compiled_1920','index','rec_db',extra_joins=extra_join)

def run( outfile, chunksize=1000000, logfile='log5.txt'):
    """Run the model on the full compare set, writing results to file."""
    total = 93000000
    for i in tqdm(range(0,total,chunksize)):
        print("start tqdm")
        
        print("getting chunk1")
        cursor = conn.cursor()
        
        cursor.execute(f"""WITH curr AS (
            SELECT * FROM compares_1910_1920 c19101920 WHERE c19101920.index1910 < {i+chunksize} AND c19101920.index1910 >= {i}
            ) 
            SELECT DISTINCT "index" as index_,* FROM compiled_1910 where "index" in (SELECT index1910 FROM curr)""")
        table = cursor.fetchallarrow()
        rec1 = table.to_pandas().set_index('index')
        
        print("getting chunk2")
        cursor.execute(f"""WITH curr AS (
            SELECT * FROM compares_1910_1920 c19101920 WHERE c19101920.index1910 < {i+chunksize} AND c19101920.index1910 >= {i}
            ) 
            SELECT DISTINCT "index" as index_, * FROM compiled_1920 where "index" in (SELECT index1920 FROM curr)""")
        table = cursor.fetchallarrow()
        rec2 = table.to_pandas().set_index('index')
     
        print("getting pairs")
        cursor.execute(f"""SELECT * FROM compares_1910_1920 c19101920 WHERE c19101920.index1910 < {i+chunksize} AND c19101920.index1910 >= {i}""")
        table = cursor.fetchallarrow()
        pairs = pd.MultiIndex.from_frame(table.to_pandas())
        pairs.names = ['','']


        comp_vecs = c.compute(pairs, rec2, rec1)
        comp_vecs.columns=['res','bp','first_jaro','last_jaro','birth_year','immigration','first_comm',
           'last_comm','marstat','mbp','fbp','rel']
                
        print("predicting")
        preds = model.predict_proba(comp_vecs) # predict matches with model
        print("Done predicting")
        

        data = pd.DataFrame({'index1910':[item[1] for item in pairs.values],'index1920':[item[0] for item in pairs.values],'link_prob':preds[:,1]})
        print("saving")
        data.to_csv(outfile,mode='a',header=None,index=False)
     

In [4]:
from recordlinkage.base import BaseCompareFeature

class eucledian_distance(BaseCompareFeature):
    def __init__(self, left_on, right_on):
        super(eucledian_distance, self).__init__(left_on, right_on)
        self.n = len(left_on)
    def _compute_vectorized(self,*args):
        s1 = args[:self.n]
        s2 = args[self.n:]
        return np.linalg.norm(np.array(s1)-np.array(s2),ord=2,axis=0)
    
class commonality_weight(BaseCompareFeature):
    def __init__(self,left_on,right_on):
        super(commonality_weight, self).__init__(left_on, right_on)
    def _compute_vectorized(self,s1,s2):
        return 1 / np.log1p((s1 + s2) / 2)
    
def get_compare_engine(drop=[]):
    exact_match_features = ['marstat','mbp','fbp','rel','first_nysiis','last_nysiis']
    exact_match_features = [feat for feat in exact_match_features if feat not in drop]
    c = rl.Compare() # declare comparison object
    if 'res' not in drop:
        c.geo('res_lat','res_lon','res_lat','res_lon',method = 'exp',scale=500)
    if 'bp' not in drop:
        c.geo('bp_lat','bp_lon','bp_lat','bp_lon', method = 'exp',scale=500)
    if 'first_jaro' not in drop:
        c.string('first','first',method = 'jarowinkler')
    if 'last_jaro' not in drop:
        c.string('last','last', method = 'jarowinkler')
    #c.string('first','first',method = 'qgram')
    #c.string('last','last', method = 'qgram')
    if 'birth_year' not in drop:
        c.numeric('birth_year','birth_year', method = 'lin', scale = 1, offset = 1)
    if 'immigration' not in drop:
        c.numeric('immigration','immigration', method = 'lin', scale = 1, offset = 1)
    
    vec_cols = [f'occ_vec{i}' for i in range(128)]
    if 'occ' not in drop:
        c.add(eucledian_distance(vec_cols,vec_cols))
    if 'comm_first' not in drop:
        c.add(commonality_weight('first_comm','first_comm'))
    if 'comm_last' not in drop:
        c.add(commonality_weight('last_comm','last_comm'))    
    for col in exact_match_features:
        c.exact(col,col)
    return c

c = get_compare_engine(drop=['occ','first_nysiis','last_nysiis'])

In [5]:
blocked_pairs = PairsDB('compares_1910_1920',['index1910','index1920'],'rec_db')
savePath = r"R:\JoePriceResearch\record_linking\projects\deep_learning\paper_RR\CensusTree_2020\final\4-predict\predictions_1910_1920_using_1900_1910.csv"
run(savePath)

  0%|                                                                               | 0/93 [00:00<?, ?it/s]

start tqdm
getting chunk1
getting chunk2
getting pairs
predicting
Done predicting
saving


  1%|▋                                                                    | 1/93 [00:59<1:30:31, 59.04s/it]

start tqdm
getting chunk1
getting chunk2
getting pairs
predicting
Done predicting
saving


  2%|█▍                                                                   | 2/93 [02:05<1:32:57, 61.29s/it]

...

 98%|█████████████████████████████████████████████████████████████████▌ | 91/93 [3:40:05<07:25, 222.85s/it]

start tqdm
getting chunk1
getting chunk2
getting pairs
predicting
Done predicting
saving


 99%|██████████████████████████████████████████████████████████████████▎| 92/93 [3:44:12<03:50, 230.04s/it]

start tqdm
getting chunk1
getting chunk2
getting pairs
predicting
Done predicting
saving


100%|███████████████████████████████████████████████████████████████████| 93/93 [3:47:57<00:00, 147.07s/it]
